## Run this first if you want NDVI/NDWI basemaps

In [1]:
import ee
ee.Authenticate()


True

## Vibes

In [8]:
%load_ext autoreload
%autoreload 2

from src.ui import GeoVibes
vibes = GeoVibes(
    duckdb_directory="gs://geovibes/databases",
    start_date="2024-01-01",
    end_date="2025-01-01",
    gcp_project="demeterlabs-gee",
    verbose=True
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Initializing GeoVibes...
✅ Earth Engine initialized with user credentials
  Found: gs://geovibes/databases/alabama_google.db
  Found: gs://geovibes/databases/java_resnet18.db
Found 2 database(s) in gs://geovibes/databases
📁 Found 2 databases in directory
🌐 Connecting to GCS database: gs://geovibes/databases/alabama_google.db
🔑 Using HMAC key authentication
✅ Database connection established successfully
⚠️  Failed to create Earth Engine boundary: None: No such file or directory
⚠️  NDVI/NDWI basemaps will be unavailable
📦 httpfs extension loaded for GCS support
🗺️  spatial extension loaded for geometry support
🔍 Detected embedding dimension: 64
🔧 Optimizing database connection...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Database optimization completed
📍 Using boundary file: gs://geovibes/geometries/alabama.geojson
Building UI...


HTML(value='\n        <style>\n        .widget-toggle-buttons button:nth-child(1).mod-active {\n            ba…

⚠️  Could not add boundary layer: [Errno 2] No such file or directory: 'gs://geovibes/geometries/alabama.geojson'
🗺️  Loading boundary layer: gs://geovibes/geometries/alabama.geojson
✅ Boundary layer added successfully


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))